# Scrape and save news from `nation.tv` to dataframe

## Getting started
1. `latest_news_id`: to the newest `news id` (or any id you want to start scraping)
2. `n_limits`: the total no. of articels to be fetched

`news id` can be found in the news url after `/content/`.

In this case, `news id` is `378805944`.

```
https://www.nationtv.tv/main/content/`378805944`?utm_source=category&utm_medium=internal_referral&utm_campaign=politics
```

## How it works

There's no way to tell a news category from `news id`, that leaves no other choices but to download and check

1. Get `news id` news
2. Check its category, if it's either การเมือง or พระราชสำหนัก, keep it, unless discard
3. Subtract `news id` by one
4. Repeat 1.-3. until getting `n_limits` news

```python
while not reaching `n_limits`:
    article = get_news_from_url(news_id)
    if article.category in ['การเมือง', 'ข่าวในพระราชสำนัก']:
        add_to_news_list(article)
    else:
        continue
    news_id -= 1
```

In [1]:
import pandas as pd

import requests

import bs4
from bs4 import BeautifulSoup

## Set up here

In [2]:
url_template = "https://www.nationtv.tv/main/content/{}"
latest_news_id = 378805944

In [3]:
save_path = "nation_politics_{}.csv"

In [4]:
allowed_categs = ['การเมือง', 'ข่าวในพระราชสำนัก']
n_limits = 4000

## Working code

In [5]:
def get_news_from_url(news_id):
    response = requests.get(url_template.format(news_id))
    html_body_str = response.text
    return html_body_str

In [6]:
body_str = get_news_from_url(latest_news_id - 999)

In [7]:
soup = BeautifulSoup(body_str, 'html.parser')

In [8]:
repr(soup)[:600]

'<!DOCTYPE html>\n\n<html dir="ltr" lang="en">\n<head>\n<title>สโมสรสมาชิก โหวต บิ๊กป้อมนั่งเก้าอี้ส.ว่ายน้ำสมัย2</title>\n<meta charset="utf-8"/>\n<meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>\n<meta content="width=device-width, height=device-height, initial-scale=1" name="viewport"/>\n<meta content="ประชุมใหญ่วิสามัญ ประจำปี2563 ,สมาคมกีฬาว่ายน้ำแห่งประเทศไทย,วาระสำคัญ,เลือกตั้งนายกสมาคมกีฬาว่ายน้ำแห่งประเทศไทย" name="keywords"/>\n<meta content=" บิ๊กป้อม   พล.อ.ประวิตร วงษ์สุวรรณ  รองนายกรัฐมนตรี ได้รับความไว้วางใจจากสโมสรสมาชิก โหวตให้เป็นนายกสมาคมกีฬาว่ายน้ำแห่งประเทศไทยสมัยที่ 2 '

In [9]:
def get_article_category(soup):
    divs = soup.findAll("div", {"class": "article-category --agents-category"})
    categ = divs[0]['data-category']
    return categ

def get_article_date(soup):
    divs = soup.findAll("div", {"class": "article-date"})
    date_str = divs[0].string
    return date_str

def get_article_preface(soup):
    divs = soup.findAll("h2", {"class": "article-preface"})
    preface_str = divs[0].string
    return preface_str

def get_article_body(soup):
    body = soup.findAll("div", {"class": "article-body"})[0]
    body_parts = []

    for elm in body.children:
        if type(elm) == bs4.element.Tag:
            content = elm.string
            if content:
                if content.strip() != "":
                    body_parts.append(content.strip())
    return "\n".join(body_parts)

In [10]:
get_article_category(soup), get_article_date(soup), get_article_preface(soup)

('กีฬา',
 '8 พ.ย. 2563',
 '" บิ๊กป้อม " พล.อ.ประวิตร วงษ์สุวรรณ รองนายกรัฐมนตรี ได้รับความไว้วางใจจากสโมสรสมาชิก โหวตให้เป็นนายกสมาคมกีฬาว่ายน้ำแห่งประเทศไทยสมัยที่ 2 โดยมีวาระในตำแหน่ง 2564-2568')

In [11]:
def parse_article(soup):
    title = soup.title.string
    date = get_article_date(soup)
    preface = get_article_preface(soup)
    body = get_article_body(soup)
    article = {
        'title': title,
        'date': date,
        'preface': preface,
        'body': body
    }
    return article

In [12]:
def scrape_nation_articles(start_news_id, n_limits=2000, allowed_categs=[]):
    news_id = start_news_id
    while len(articles) < n_limits:
        if len(articles) % 100 == 0 and articles:
            print("{} articles fetched".format(len(articles)))
        html_body_str = get_news_from_url(news_id)
        soup = BeautifulSoup(html_body_str, 'html.parser')
        try:
            categ = get_article_category(soup)
        except:
            print("Not found. Cannot find title")
            news_id -= 1
            continue
        
        if categ in allowed_categs:
            try:
                article = parse_article(soup)
            except:
                print("Article parse error")
                news_id -= 1
                continue
            article['id'] = news_id
            article['category'] = categ
            articles.append(article)
        
        news_id -= 1

In [ ]:
articles = []
articles = scrape_nation_articles(latest_news_id, n_limits=n_limits, allowed_categs=allowed_categs)

In [14]:
articles[0]

{'title': 'แร็พประเทศกูมี ปล่อยเพลงใหม่ปฏิรูปฉากหลังเป็นม็อบ ยอดวิวพุ่งติดอันดับหนึ่ง',
 'date': '14 พ.ย. 2563',
 'preface': 'ดูกันหรือยังเพลงและมิวสิกวีดีโอใหม่ของแร็พประเทศกูมี กับเพลง "ปฏิรูป"ที่เนื้อเพลงยังคงจิกกัดสะท้อนสังคมในมุมมองผู้สื่อสารให้เห็น แถมยังมีฉากหลังเป็นเป็นภาพการชุมนุมจริงของม็อบ "ราษฎร" ทั้งบริเวณอนุสาวรีย์ประชาธิปไตย แยกราชประสงค์ และอนุสาวรีย์ชัยสมรภูมิ ตอนนี้กลายเป็นเพลงที่มาแรงติดอันดังหนึ่งบนยูทูปป',
 'body': 'ส่วนเนื้อเพลงนั้นมีเนื้อหาว่า\nเรื่องประยุทธกูแ-งไม่อินแต่เรื่องจริง-very goodที่บ้าน-ไม่มีปฏิทินเพราะตามจริงต้องการปฏิรูปมึงเอาเงินภาษีไปกินพวกกูเลยจำเป็นต้องออกมาพูดก้มกราบไม่ค่อยจะชินยกสามนิ้วพวก-อ่ะชู\nฟังท่อนนี้ละเสียงคุ้นๆประเทศกูมีท่อนแรกอ่ะ-พี่น้องพวกเราโดนยิงและคนสั่งมันชื่อ----พวกเรามีแต่ประชาชน..พวกส้นตีนมีแต่---รอวันศักดินาพินาศประชาราษฏรโห่ร้องไชโย\nเรียกร้องประชาธิปไตยออกมาชุมนุมกันอย่างสันติข้อเสนอต้องย้ำจุดยืนตั้งแต่10จนถึง3 2 1สู้ด้วยเสรีภาพความคิดไม่ใช่ความผิดที่ต้องโดนจับบุกล้อมหน้าบ้านจับกูแล้วมาแจ้งยุยงปลุกปั่น\nออกไปเอ้ยเอ้ยออกไปเอ

In [15]:
df_nation_news = pd.DataFrame(articles)

In [16]:
def normalize_date(date_th_str):
    day, month, year = date_th_str.split()
    months_th = ["มค", "กพ" , "มีค" , "เมย" , "พค", "มิย", 
              "กค", "สค", "กย", "ตค", "พย", "ธค"]
    month_num = months_th.index(month.replace(".", "")) + 1
    year_ad = int(year) - 543
    date_fmt = "{}/{}/{}".format(year_ad, str(month_num).zfill(2), day.zfill(2))
    return date_fmt

In [17]:
df_nation_news['date'] = df_nation_news['date'].apply(normalize_date)

In [18]:
df_nation_news.head(3)

,body,category,date,id,preface,title
0,ส่วนเนื้อเพลงนั้นมีเนื้อหาว่า\nเรื่องประยุทธกู...,การเมือง,2020/11/14,378805944,ดูกันหรือยังเพลงและมิวสิกวีดีโอใหม่ของแร็พประเ...,แร็พประเทศกูมี ปล่อยเพลงใหม่ปฏิรูปฉากหลังเป็นม...
1,ส่วนที่บริเวณบนฟุตบาธริมถนนมีแนวกั้นสังกะสีมาว...,การเมือง,2020/11/14,378805943,ตำรวจซักซ้อมวางแนวกั้นแบบเร่งด่วน นำรถเมล์มาจอ...,ผวาม็อบ!! ตำรวจซ้อมวางแนวกั้น ปิดราชดำเนินกลางดึก
2,โครงการพระราชดำริฝนหลวง เกิดขึ้นจากพระราชดำริส...,ข่าวในพระราชสำนัก,2020/11/14,378805928,"14 พฤศจิกายน ของทุกปี คือ ""วันพระบิดาแห่งฝนหลว...",14 พฤศจิกายน วันพระบิดาแห่งฝนหลวง


In [19]:
df_nation_news.to_csv(save_path.format(len(articles)), index=False)